In [83]:
import numpy as np
from ctypes import c_int32
import csv

# CGRA from left to right, top to bottom
N_ROWS = 4
N_COLS = 4
INSTR_SIZE = N_ROWS+1
MAX_COL = N_COLS - 1
MAX_ROW = N_ROWS - 1

MAX_32b = 0xFFFFFFFF


srcs    = ['ZERO', 'SELF', 'RCL', 'RCR', 'RCT', 'RCB',  'R0', 'R1', 'R2', 'R3', 'IMM']
dsts    = ['SELF', 'RCL', 'RCR', 'RCT', 'RCB','R0', 'R1', 'R2', 'R3']

class INSTR:
    def __init__( self,matrix):
        self.time = matrix[0][0]                        # ToDo: Fix how we assign this length
        self.ops = [[ matrix[r+1][c] for c in range(N_COLS)] for r in range(N_ROWS)]

def csv_parse( f ):
    data = []
    read = csv.reader(f)
    for row in read:
        data.append(row)
    instrs = int(len(data)/( INSTR_SIZE  )) # Always have a CSV with as many csv-columns as CGA-columns. Each instruction starts with the instruction timestamp i nthe first column. The next instruction must be immediately after the last row of this instruction.
    return [ INSTR( data[r_i*INSTR_SIZE:(r_i+1)*INSTR_SIZE][0:] ) for r_i in range(instrs) ]

class CGRA:
    def __init__( self, kernel, memory, inputs, outputs ):
        self.cells      = [[ PE( self, c,r) for r in range(N_ROWS)] for c in range(N_COLS)]
        self.instrs     = csv_parse( kernel )
        self.memory     = memory
        self.inputs     = inputs
        self.outputs    = outputs
        self.instr2exec = 0
        self.load_idx   = np.zeros( N_COLS )
        self.store_idx  = np.zeros( N_COLS ) 
        self.exit       = False

    def step( self):
        for r in range(N_ROWS):
            for c in range(N_COLS):
                self.cells[r][c].update()

        for r in range(N_ROWS):
            for c in range(N_COLS):
                op =  self.instrs[self.instr2exec].ops[r][c]
                b ,e = self.cells[r][c].exec( op )
                if b != 0: self.instr2exec = b - 1 #To avoid more logic afterwards
                if e != 0: self.exit = True
            outs = [ self.cells[r][i].out for i in range(N_COLS) ]
            print(outs)
        self.instr2exec += 1
        return self.exit

    def get_neighbour_address( self, r, c, dir ):
        n_r = r
        n_c = c
        if dir == "RCL": n_c = c - 1 if c > 0 else MAX_COL
        if dir == "RCR": n_c = c + 1 if c < MAX_COL else 0
        if dir == "RCT": n_r = r - 1 if r > 0 else MAX_ROW
        if dir == "RCB": n_r = r + 1 if r < MAX_ROW else 0
        return n_r, n_c

    def get_neighbour_out( self, r, c, dir ):
        n_r, n_c = self.get_neighbour_address( r, c, dir )
        return self.cells[n_r][n_c].get_out() #fix this. This does not know what cgra is
    
    def get_neighbour_flag( self, r, c, dir, flag ):
        n_r, n_c = self.get_neighbour_address( r, c, dir )
        return self.cells[n_r][n_c].get_flag( flag )

    def load_direct( self, c ):
        ret = self.inputs[ c, self.load_idx[c] ]
        self.load_idx[c] += 1
        return ret
    
    def store_direct( self, c, val ):
        self.outputs[c][ self.store_idx[c] ] = val
        self.load_idx[c] += 1

class PE:
    def __init__( self, parent, row, col ):
        self.parent = parent
        self.row = row
        self.col = col
        self.flags      = { "sign"   : 0,
                            "zero"   : 0,
                            "branch" : 0,
                            "exit"   : 0}
        self.instr      = ""
        self.old_out    = 0
        self.out        = 0
        self.regs       = {'R0':0, 'R1':0, 'R2':0, 'R3':0 }

    def get_out( self ):
        return self.old_out
    
    def get_flag( self, flag ):
        return self.flags[flag]

    def fetch_out( self, val):
        if val.isnumeric():
            return int(val)
        if val == 'ROUT':
            return int( self.old_out)
        if val in self.regs:
            return int( self.regs[val])
        return int(self.parent.get_neighbour_out( self.row, self.col, val ))
    
    def fetch_flag( self, flag, dir ):
        if dir == 'ROUT':
            return int( self.old_out)
        return int(self.parent.get_neighbour_flag( self.row, self.col, dir, flag ))

    def exec( self,  instr ):
        self.run_instr(instr)
        return self.flags["branch"], self.flags["exit"]

    def update( self):
        self.old_out = self.out
        self.flags["zero"] = 1 if self.out == 0 else 0
        self.flags["sign"] = 1 if self.out <  0 else 0

    def run_instr( self, instr):
        instr   = instr.replace(',', ' ') # Remove the commas so we can speparate arguments by spaces
        instr   = instr.split()        # Split into chunks
        try:
            op      = instr[0] 
        except:
            op = instr

        if op in self.ops_arith:
            des     = instr[1]
            val1    = self.fetch_out( instr[2] )
            val2    = self.fetch_out( instr[3] )
            ret     = self.ops_arith[op]( val1, val2)
            if des in self.regs: self.regs[des] = ret
            else: self.out = ret

        elif op in self.ops_cond:
            des     = instr[1]
            val1    = self.fetch_out( instr[2] )
            val2    = self.fetch_out( instr[3] )
            src     = instr[4]
            ret     = self.ops_cond[op]( val1, val2, src )
            if des in self.regs: self.regs[des] = ret
            else: self.out = ret

        elif op in self.ops_branch:
            val1    = self.fetch_out( instr[1] )
            val2    = self.fetch_out( instr[2] )
            branch  = instr[3]
            self.ops_branch[op]( val1, val2, branch )

        elif op in self.ops_dir:
            des = instr[1]
            ret = self.ops_dir[op]()
            if des in self.regs: self.regs[des] = ret
            else: self.out = ret

        elif op in self.ops_indir:
            des = instr[1]
            add = instr[2]
            ret = self.ops_indir[op]( add )
            if des in self.regs: self.regs[des] = ret
            else: self.out = ret

        elif op in self.ops_nop:
            pass # Intentional

        elif op in self.ops_jump:
            self.flags['branch'] = val1
            
        elif op in self.ops_exit:
            self.flags['exit'] = 1

    def sadd( val1, val2 ):
        return c_int32( val1 + val2 ).value

    def ssub( val1, val2 ):
        return c_int32( val1 - val2 ).value

    def smul( val1, val2 ):
        return c_int32( val1 * val2 ).value

    def fxpmul( val1, val2 ):
        print("Better luck next time")
        return 0

    def slt( val1, val2 ):
        return c_int32(val1 << val2).value
    
    def srt( val1, val2 ):
        return c_int32(val1 >> val2).value # ToDo: Fix this one

    def sra( val1, val2 ):
        return c_int32(val1 >> val2).value

    def lor( val1, val2 ):
        return c_int32( val1 | val2).value

    def land( val1, val2 ):
        return c_int32( val1 & val2).value

    def lxor( val1, val2 ):
        return c_int32( val1 ^ val2).value

    def lnand( val1, val2 ):
        return c_int32( ~( val1 & val2 ) & MAX_32b ).value
        
    def lnor( val1, val2 ):
         return c_int32( ~( val1 | val2 ) & MAX_32b ).value

    def lxnor( val1, val2 ):
        return c_int32( ~( val1 ^ val2 ) & MAX_32b ).value

    def bsfa( self, val1, val2, src):
        flag = self.fetch_flag( self.row, self.col, src, 'sign') 
        return val1 if flag == 1 else val2
        
    def bzfa( self,  val1, val2, src):
        flag = self.fetch_flag( self.row, self.col, src, 'zero') 
        return val1 if flag == 1 else val2
    
    def beq( self,  val1, val2, branch ):
        self.flags['branch'] = branch if val1 == val2 else self.flags['branch']       

    def bne( self,  val1, val2, branch ):
        self.flags['branch'] = branch if val1 != val2 else self.flags['branch'] 

    def bge( self,  val1, val2, branch ):
        self.flags['branch'] = branch if val1 >= val2 else self.flags['branch'] 

    def lwd( self, dest ):
        return self.parent.load_direct( self.col )

    def swd( self, val ):
        self.parent.store_direct( self.col, val )

    def lwi( self, dest, address ):
        pass

    def swi( self, src, address ):
        pass 


    ops_arith   = { 'SADD'      : sadd,
                    'SSUB'      : ssub,
                    'SMUL'      : smul,
                    'FXPMUL'    : fxpmul, 
                    'SLT'       : slt,
                    'SRT'       : srt,
                    'SRA'       : sra,
                    'LOR'       : lor,
                    'LAND'      : land,
                    'LXOR'      : lxor,
                    'LNAND'     : lnand, 
                    'LNOR'      : lnor, 
                    'LXNOR'     : lxnor }
    
    ops_cond    = { 'BSFA'      : bsfa,
                    'BZFA'      : bzfa }

    ops_branch  = { 'BEQ'       : beq,
                    'BNE'       : bne,
                    'BGE'       : bge }

    ops_dir     = { 'LWD'       : lwd,
                    'SWD'       : swd }
    
    ops_indir   = { 'LWI'       : lwi,
                    'SWI'       : swi,}

    ops_nop     = { 'NOP'       : ''  }
    ops_jump    = { 'JUMP'      : '' }
    ops_exit    = { 'EXIT'      : '' }

    

def run():
    ker = open("instructions.csv")
    mem = open("memory.csv")
    inp = open("inputs.csv")
    oup = open("outputs.csv")
    
        


    cgra = CGRA( ker, mem, inp, oup )
    while not cgra.step():
        print("-------")
    
    mem.close()
    ker.close()
    inp.close()
    oup.close()
    print("\n\nEND")


run()

[5, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
-------
[5, 7, 0, 0]
[6, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]


END
